This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [131]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [10]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
def get_request(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("HTTP Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("Something went wrong",err)
    else:
        return response.json()  # convert JSON response to Python dictionary

# Custom Parameters
start_date = '2017-01-01'
end_date = '2017-12-31'
url = f"https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}&start_date={start_date}&end_date={end_date}"

# Perform GET Request
response_dict = get_request(url)

In [11]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

# Result is multi-nested, we'll need everything in "dataset_data"
response_dict.keys()

dict_keys(['dataset_data'])

In [12]:
# Modify data to return everything in "dataset_data"
response_dict = response_dict['dataset_data']
response_dict.keys() # Check the keys

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [14]:
# Extract the column names
column_names = response_dict['column_names']
column_names

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [17]:
# Exercise 1 - Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
# Make sure data is 2017
print(f"Start Date: {response_dict['start_date']}, End Date: {response_dict['end_date']}")

Start Date: 2017-01-01, End Date: 2017-12-31


In [18]:
# Exercise 2 - Convert the returned JSON object into a Python dictionary.
    # Result is already converted in "get_request" function
print(type(response_dict))

<class 'dict'>


In [26]:
# Get the Data
data = response_dict['data']
# Test print first 3 rows
print(data[0:3])

[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None]]


In [68]:
# Function to get non-null values from a column in a 2D data
def get_non_null_values_from_column_x(data, col_name):
    col_index = column_names.index(col_name)
    return [row[col_index] for row in data if row[col_index] is not None]

In [84]:
# Exercise 3 - Calculate what the highest and lowest opening prices were for the stock in this period.
open_prices = get_non_null_values_from_column_x(data, 'Open')

print(f'Highest vs Lowest Opening Prices: {max(open_prices)} EUR vs {min(open_prices)} EUR')

Highest vs Lowest Opening Prices: 53.11 EUR vs 34.0 EUR


In [89]:
# Exercise 4 - What was the largest change in any one day (based on High and Low price)?
one_day_price_diff_high_vs_low = []
for row in data:
    high_col_index = column_names.index('High')
    low_col_index = column_names.index('Low')
    one_day_price_diff_high_vs_low.append(row[high_col_index] - row[low_col_index])
    
print(f'Largest Change in one-day (based on High - Low Price): {round(max(one_day_price_diff_high_vs_low), 4)} EUR')

Largest Change in one-day (based on High - Low Price): 2.81 EUR


In [120]:
# Exercise 5 - What was the largest change between any two days (based on Closing Price)?
closing_prices = get_non_null_values_from_column_x(data, 'Close')
    
print(f'Largest Change Between Any Two Days (based on Closing Price): {round(max(closing_prices) - min(closing_prices), 4)} EUR')

Largest Change Between Any Two Days (based on Closing Price): 19.03 EUR


In [125]:
# Exercise 6 - What was the average daily trading volume during this year?
daily_trading_volume = get_non_null_values_from_column_x(data, 'Traded Volume')
avg_daily_trading_volume = sum(daily_trading_volume) / len(daily_trading_volume)

print(f'Largest Change Between Any Two Days (based on Closing Price): {round(avg_daily_trading_volume)}')

Largest Change Between Any Two Days (based on Closing Price): 89124


In [126]:
# Exercise 7 - (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

def get_median(list_of_numbers):
  """This function finds the median of a list of numbers.

  Args:
    list_of_numbers: A list of numbers.

  Returns:
    The median of the list of numbers.
  """

  list_of_numbers.sort()

  # If there is an even number of elements, the median is the average of the middle two.

  if len(list_of_numbers) % 2 == 0:
    median = (list_of_numbers[len(list_of_numbers) // 2] +
              list_of_numbers[len(list_of_numbers) // 2 - 1]) / 2

  # If there is an odd number of elements, the median is the middle element.

  else:
    median = list_of_numbers[len(list_of_numbers) // 2]

  return median

median_trading_volume = get_median(daily_trading_volume)
print(f'Largest Change Between Any Two Days (based on Closing Price): {round(median_trading_volume)}')

Largest Change Between Any Two Days (based on Closing Price): 76286
